In [44]:
%%writefile app.py
import pandas as pd
from dash import Dash,html,dcc, Input,Output

from plotly.express import line

PATH = 'formatted_data.csv'
COLORS = {
    "primary":"#FFDBDB",
    "secondary": "#EB9898",
    "font": "#522A61"
}

data = pd.read_csv(PATH)
data = data.sort_values(by='date')

app = Dash()

def generate_figure(chart_data):
    line_chart = line(chart_data,x='date',y='Sales',title='Sales of Pink Morsel')
    line_chart.update_layout(
        plot_bgcolor = COLORS['secondary'],
        paper_bgcolor = COLORS['primary'],
        font_color = COLORS['font']
    )
    return line_chart

visualization = dcc.Graph(
    id='visualization',
    figure=generate_figure(data)
)

header = html.H1(
    "Sales of Pink Morsel Visualizer",
    id='header',
    style={
        "background-color":COLORS["secondary"],
        'color':'black',
        'boder-radius':'20px'
    }
)

region_picker = dcc.RadioItems(
    ['north','east','south','west','all'],
    'north',
    id='region_picker',
    inline=True
)

region_picker_wrapper = html.Div(
    [region_picker],style={
        'font-size':'150%'
    }
)

@app.callback(
    Output(visualization,"figure"),
    Input(region_picker,"value")
)
def update_graph(region):
    if region == 'all':
        filtered_data =  data
    else:
        filtered_data = data[data['region'] == region]

    figure=generate_figure(filtered_data)
    
    return figure


app.layout = html.Div(
    [header,visualization,region_picker_wrapper],
    style={
        'textAlign':'center',
        'background-color':COLORS['primary'],
        'boder-radius':'20px'
    }
)

if __name__ =='__main__':
    app.run()

Writing app.py


In [46]:
%%writefile test.py
from app import app


def test_header_exists(dash_duo):
    dash_duo.start_server(app)
    dash_duo.wait_for_element("#header", timeout=10)


def test_visualization_exists(dash_duo):
    dash_duo.start_server(app)
    dash_duo.wait_for_element("#visualization", timeout=10)


def test_region_picker_exists(dash_duo):
    dash_duo.start_server(app)
    dash_duo.wait_for_element("#region_picker", timeout=10)

Writing test.py


In [48]:
!pytest test.py

============================= test session starts =============================
platform win32 -- Python 3.12.6, pytest-9.0.2, pluggy-1.6.0
rootdir: c:\FORAGE\quantium
plugins: dash-3.3.0
collected 3 items

test.py ...                                                              [100%]

============================== warnings summary ===============================
test.py: 21 warnings
  C:\VIRTUAL_ENV\universal_venv\Lib\site-packages\selenium\webdriver\remote\remote_connection.py:390: DeprecationWarning:
  
  HTTPResponse.getheader() is deprecated and will be removed in urllib3 v2.1.0. Instead use HTTPResponse.headers.get(name, default).

test.py: 21 warnings
  C:\VIRTUAL_ENV\universal_venv\Lib\site-packages\selenium\webdriver\remote\remote_connection.py:391: DeprecationWarning:
  
  HTTPResponse.getheader() is deprecated and will be removed in urllib3 v2.1.0. Instead use HTTPResponse.headers.get(name, default).

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
